## Filter Action-Object Pairs

In [1]:
import pandas as pd
from collections import Counter
import spacy
from spacy.matcher import Matcher
from tqdm import tqdm
from IPython.display import display, HTML
tqdm.pandas()


# Load the English model
nlp = spacy.load("en_core_web_lg")

In [2]:
df = pd.read_parquet('../../data/processed/targets/avocado_train_targets_exploded.parquet', engine='fastparquet')
df = df.rename(columns={'targets': 'target'})

In [3]:
df.shape

(1205714, 19)

In [4]:
# Remove empty entries
df_filtered = df[~df['action_object_pairs'].apply(lambda x: x is None or (isinstance(x, list) and len(x) == 0))]

In [5]:
df_filtered.shape

(722472, 19)

### Inspect Action-Object-Pairs results

In [6]:
def get_distinct_words_df(df):
    all_words = [word for sublist in df['action_object_pairs'] for word in sublist]

    # Use Counter to count occurrences of each word
    word_counts = Counter(all_words)

    # Convert to DataFrame (optional, if you want to keep it in tabular form)
    distinct_words_df = pd.DataFrame(word_counts.items(), columns=['action_object_pairs', 'count'])
    distinct_words_df = distinct_words_df.sort_values(by='count', ascending=False)
    #display(HTML(distinct_words_df.head(n).to_html()))
    return distinct_words_df

In [7]:
get_distinct_words_df(df_filtered).head(20)

,action_object_pairs,count
20,start_Server,12831
63,send_it,12052
177,have_questions,11024
64,send_message,10704
301,send_email,8186
19,fail_Message,8065
18,start_Failures,6983
376,start_Occurrences,6428
395,call_me,6177
103,thank_you,4472


### Inspect Messages


| Action-Object-Pair    | Count |
|-------------|-------------|
 start_Server |	12831
send_it |	12052
have_questions |	11024
send_message |	10704
send_email |	8186
fail_Message |	8065
start_Failures |	6983
start_Occurrences |	6428
call_me	| 6177
thank_you | 4472
contact_me | 3995
post_message | 3790
miss_UNIVERSE | 3742
do_what | 3584
send_mail | 3490
start_occurrence | 3152
do_it | 2816
give_call | 2642
need_help | 2541
unsubscribe_mailto | 2468

In [8]:
# Define a function to inspect entries for a given action-object pair
def inspect_action_object_pair(pair_to_inspect, entry, print_extracted_text=True, print_target=True):
    test = df_filtered[df_filtered['action_object_pairs'].apply(lambda x: pair_to_inspect in x)]
    if print_extracted_text:
        print(test.iloc[entry]['extracted_text'])
    if print_target:
        if print_extracted_text:
            print('----------------------')
    print(test.iloc[entry]['target'])
    del test

In [9]:
inspect_action_object_pair('send_e', 1,print_extracted_text=False)

I might have sent the e-mail to the wrong person because you mentioned before that we should have a meeting with Mark


## Entry Removal

After inspecting the most-frequently occurring action-object-pairs, the following ones will not be considered further:

| Action-Object-Pair    | Reason |
|-------------|-------------|
|fail_Message|belongs to error message and thus does not contain a humanly intent|
|start_Failures|belongs to error message and thus does not contain a humanly intent|
|start_Occurrences|belongs to error message and thus does not contain a humanly intent|
|post_message|belongs to an automated message|
|unsubscribe_mailto|belongs to an automated message|
|miss_UNIVERSE|belongs to an automated message|
|start_occurrence|belongs to an automated message|

also the extremeprogramming unsubscribe message is being filtered from the original dataframe, as well as java messages

In [39]:
entries_to_remove = [
    'fail_Message',
    'start_Failures',
    'start_Occurrences',
    'post_message',
    'unsubscribe_mailto',
    'miss_UNIVERSE',
    'start_occurrence',
    'set_sender'
]

# Remove these entries
df_filtered = df_filtered[~df_filtered['action_object_pairs'].apply(lambda x: any(item in entries_to_remove for item in x))]

In [40]:
df_filtered.shape

(686512, 19)

In [41]:
test_string = "Unexpected flow reached by the server"

df_filtered = df_filtered[~df_filtered['extracted_text'].apply(lambda x: test_string in str(x))]


In [42]:
df_filtered.shape

(686512, 19)

In [13]:
# Save the cleaned model

df_filtered.to_parquet('../../data/processed/targets/avocado_train_targets_exploded_cleaned.parquet')

In [43]:
get_distinct_words_df(df_filtered).head(20)

,action_object_pairs,count
170,have_questions,11024
294,send_email,8182
58,send_it,6188
387,call_me,6177
59,send_message,4856
97,thank_you,4426
227,contact_me,3994
402,do_what,3567
666,send_mail,3490
146,do_it,2806


In [44]:
# Filter to keep only unique message IDs
unique_df = df_filtered[~df_filtered['messageid'].duplicated(keep=False)]

entry = 0
print(unique_df.iloc[entry]['extracted_text'])
print('-------------------------------------')
print(unique_df.iloc[entry]['target'])
print('-------------------------------------')
print(unique_df.iloc[entry]['action_object_pairs'])

Venk,

Congratulations. This is wonderful.

Let me know if I can help in any way.

best

raman

At 08:17 PM 7/20/00 -0700, you wrote:
>
>Friends,
>
>I am happy to share with you our excitement at being selected as one of the
>top 25 b-to-b companies by the Forbes magazine.
>
>http://www.forbesbest.com/b2b/asp/industry.asp
>

>
>

-------------------------------------
Let me know if I can help in any way.

best

raman

At 08:17 PM 7/20/00 -0700, you wrote:
>
>Friends,
>
>
-------------------------------------
['write_raman']


### Group Pairs

In [49]:
target_strings = 'send_'
df_send = df_filtered[df_filtered['action_object_pairs'].apply(lambda x: isinstance(x, list) and any(target_strings in item for item in x))]

In [50]:
df_send.head()

,messageid,sender_access,outlook_sender_name,outlook_recipient_name,sentto_address,cc_address,subject,processed_subject,sent_date,arrival_date,body_file_path,body_content,in_reply_to,Keywords,extracted_text,preceding_conversation,sentences,target,action_object_pairs
2,<01370F5B01AAD411BE8100D0B744338C24F5A0@HQEXCH>,EX,Jackie Valle,None,Toshiya Otani,Jackie Valle,Tickets to Tokyo,Tickets to Tokyo,2001-02-22T19:38:41Z,2001-02-22T19:38:41Z,text/096/096-001089-EM.txt,"From: ""Jackie Valle"" <jackie.valle@avocadoit.c...",None,None,"Hi,\r\n\r\nToday I received your tickets to To...",None,"[Hi,\r\n\r\nToday I received your tickets to T...",Being that she told you and I [on the phone]\r...,"[tell_you, send_them]"
34,<FC38A9406AA4D411AB62009027DE9DA502214BAD@ntma...,EX,Arun Kalasapudi,Shailesh Kheny,Shailesh Kheny,None,FW: Email Update,Email Update,2001-09-07T13:42:58Z,2001-09-07T13:41:53Z,text/232/232-001946-EM.txt,Received: from mailer.avocadoit.com (10.7.1.4 ...,None,None,X-Authentication-Warning: bayunix.bayrentals.c...,From: Bay Rentals <support@bayrentals.com>\r\n...,[X-Authentication-Warning: bayunix.bayrentals....,* * *\r\n Please do not send EMAIL for ch...,"[send_EMAIL, delay_service]"
70,<m27kum7vak.fsf@zip.local.thomases.com>,SMTP,Dave Thomas,Madhava Gullapalli,extremeprogramming@yahoogroups.com,None,Re: [XP] Arguments for Always Pair Programming,[XP] Arguments for Always Pair Programming,2001-09-25T22:36:03Z,2001-09-25T22:43:01Z,text/142/142-015683-EM.txt,Received: from mailer.avocadoit.com (10.7.1.4 ...,None,None,"""Glew, Andy"" <andy.glew@intel.com> writes:\r\n...",None,"[""Glew, Andy"" <andy.glew@intel.com> writes:\r\...","Dave\r\n\r\nTo Post a message, send it to: e...","[send_it, send_message]"
87,<FC38A9406AA4D411AB62009027DE9DA5018F4EFA@ntma...,EX,Richard Yoza,DA,'neel.k.sata@accenture.com'; Shailesh Kheny,Support; aimee.e.grevemberg@accenture.com; dev...,RE: Worldcom NOE AvocadoIT Error,Worldcom NOE AvocadoIT Error,2001-07-25T19:53:42Z,2001-07-25T19:53:11Z,text/062/062-056161-EM.txt,"From: ""Richard Yoza"" <ryoza@avocadoit.com>\r\n...",<OF683AAED3.D6A69485-ON85256A94.006A8314@accen...,None,"Neel,\r\nTo help us offer more efficient servi...",None,"[Neel,\r\nTo help us offer more efficient serv...","Neel,\r\nTo help us offer more efficient servi...","[offer_service, send_requests, note_change]"
90,<FC38A9406AA4D411AB62009027DE9DA5018F4EFA@ntma...,EX,Richard Yoza,DA,'neel.k.sata@accenture.com'; Shailesh Kheny,Support; aimee.e.grevemberg@accenture.com; dev...,RE: Worldcom NOE AvocadoIT Error,Worldcom NOE AvocadoIT Error,2001-07-25T19:53:42Z,2001-07-25T19:53:11Z,text/062/062-056161-EM.txt,"From: ""Richard Yoza"" <ryoza@avocadoit.com>\r\n...",<OF683AAED3.D6A69485-ON85256A94.006A8314@accen...,None,"Neel,\r\nTo help us offer more efficient servi...",None,"[Neel,\r\nTo help us offer more efficient serv...",Send any additional questions or concerns thro...,"[send_questions, send_Richard]"


In [51]:
df_send.shape

(54000, 19)

In [53]:
df_send.to_parquet('../../data/processed/intents/send.parquet')

In [57]:
print(df_send.iloc[10]['target'])

Well, since I did not send the official invite, here's one from the Bday
boy.


#### obsolete code

In [15]:
df_distinct_words = get_distinct_words_df(df_filtered)

In [ ]:
df_distinct_words.head(60)

In [15]:
# Define function to aggregate rows for a given word. If an action-object-pair contains the given word, entries and their occurrence-count will be aggregated.

def aggregate_rows(df, word_to_group):
    # Step 1: Filter rows where 'action_object_pairs' contains the word to group
    filtered_df = df[df['action_object_pairs'].str.contains(word_to_group, case=False, na=False)]

    # Step 2: Aggregate the filtered entries
    new_entry = pd.DataFrame({
        'action_object_pairs': [filtered_df['action_object_pairs'].tolist()],  # Aggregate label
        'count': [filtered_df['count'].sum()]                   # Sum the counts
    })

    # Step 3: Remove the filtered entries from the original DataFrame
    df = df[~df['action_object_pairs'].str.contains(word_to_group, case=False, na=False)]

    # Step 4: Append the new entry to the original DataFrame
    df = pd.concat([df, new_entry], ignore_index=True)

    # Sort the DataFrame by 'count' column in descending order
    return df.sort_values(by='count', ascending=False).reset_index(drop=True)


In [93]:
# Group for words "send", "call", "meeting", "join", "thank", "do", "receive", "provide", "discuss", "changes", "need"
df_distinct_words = aggregate_rows(df_distinct_words,'see')

In [ ]:
df_distinct_words.head(60)

In [ ]:
df_distinct_words.shape